# Workshop: Workspace Setup, Data Import & Exploration

**Cel szkoleniowy:** Praktyczne opanowanie konfiguracji workspace, importowania danych z różnych formatów oraz podstawowych operacji eksploracyjnych.

**Zakres tematyczny:**
- Konfiguracja workspace i klastrów
- Wczytywanie różnych formatów danych (CSV, JSON, Parquet)
- Podstawowa eksploracja danych
- Konstruowanie schematów manualnie
- Analiza braków danych i wartości unikalnych

**Czas trwania:** 90 minut

## Kontekst i wymagania

- **Dzień szkolenia**: Dzień 1 - Fundamentals & Exploration
- **Typ notebooka**: Workshop
- **Wymagania techniczne**:
  - Databricks Runtime 13.0+ (zalecane: 14.3 LTS)
  - Unity Catalog włączony
  - Uprawnienia: CREATE TABLE, CREATE SCHEMA, SELECT, MODIFY
  - Klaster: Standard z minimum 2 workers

## Wprowadzenie do warsztatu

W tym warsztacie będziesz pracować z rzeczywistymi danymi KION:
- **Customers** (customers.csv) - dane klientów
- **Orders** (orders_batch.json) - zamówienia
- **Products** (products.parquet) - produkty

### Zadania do wykonania:
1. Konfiguracja środowiska i zmiennych
2. Import danych z CSV, JSON, Parquet
3. Konstrukcja schematów manualnie
4. Eksploracja danych: statystyki, braki, unikalne wartości
5. Analiza jakości danych

### Kryteria sukcesu:
- Wszystkie 3 datasety poprawnie wczytane
- Schematy zdefiniowane manualnie i zastosowane
- Kompletna analiza eksploracyjna przeprowadzona
- Zidentyfikowane problemy jakości danych

## Wstęp teoretyczny

**Cel sekcji:** Zrozumienie podstaw pracy z danymi w Databricks Lakehouse

**Podstawowe pojęcia:**
- **Workspace**: Środowisko Databricks zawierające notebooks, klastry i dane
- **Cluster**: Zbiór maszyn wirtualnych przetwarzających dane
- **DataFrame**: Rozproszona kolekcja danych zorganizowana w kolumny
- **Schema**: Struktura danych definiująca nazwy kolumn i typy danych
- **Format danych**: CSV (text), JSON (semi-structured), Parquet (columnar binary)

**Dlaczego to ważne?**
Poprawne wczytanie i eksploracja danych to fundament każdego pipeline'u ETL. Zrozumienie schematów, formatów i metod eksploracji pozwala na wczesne wykrycie problemów jakości danych.

## Inicjalizacja środowiska

Uruchom skrypt inicjalizacyjny dla per-user izolacji katalogów i schematów:

In [0]:
%run ../../00_setup

## Konfiguracja

Zdefiniuj zmienne specyficzne dla warsztatu:

In [0]:
from pyspark.sql import functions as F
from pyspark.sql.types import *

# Ścieżki do plików danych (już zdefiniowane)
CUSTOMERS_CSV = f"{DATASET_BASE_PATH}/customers/customers.csv"
ORDERS_JSON = f"{DATASET_BASE_PATH}/orders/orders_batch.json"
PRODUCTS_PARQUET = f"{DATASET_BASE_PATH}/products/products.parquet"

# Wyświetl kontekst użytkownika
print("=== Kontekst użytkownika ===")
print(f"Katalog: {CATALOG}")
print(f"Schema Bronze: {BRONZE_SCHEMA}")
print(f"Użytkownik: {raw_user}")
print(f"\n=== Ścieżki do danych ===")
print(f"Customers CSV: {CUSTOMERS_CSV}")
print(f"Orders JSON: {ORDERS_JSON}")
print(f"Products Parquet: {PRODUCTS_PARQUET}")

---

## Zadanie 1: Import danych CSV (20 min)

### Cel:
Wczytaj dane klientów z pliku CSV, najpierw z automatycznym wykrywaniem schematu, potem z ręcznie zdefiniowanym schematem.

### Instrukcje:
1. Wczytaj `customers.csv` z opcją `inferSchema=True`
2. Wyświetl schemat i 5 pierwszych rekordów
3. Policz liczbę rekordów
4. Zdefiniuj schemat manualnie (StructType)
5. Wczytaj ponownie używając ręcznego schematu
6. Porównaj schematy

### Oczekiwany rezultat:
- DataFrame z danymi klientów
- Schemat zawierający kolumny: customer_id (int), customer_name (string), email (string), country (string), registration_date (timestamp)

### Wskazówki:
- Użyj `spark.read.format("csv").option("header", "true")`
- Do ręcznego schematu użyj: `StructType`, `StructField`, `IntegerType`, `StringType`, `TimestampType`

In [0]:
# Przykład: Wczytanie pliku CSV z klientami
# Ta komórka jest już gotowa - przeanalizuj kod i uruchom ją

customers_df = (
    spark.read
    .format("csv")
    .option("header", "true")
    .option("inferSchema", "true")
    .load(CUSTOMERS_CSV)
)

# Wyświetl schemat i pierwsze wiersze
print("=== Schema ===")
customers_df.printSchema()

print("\n=== Przykładowe dane (5 wierszy) ===")
display(customers_df.limit(5))

# Policz rekordy
count = customers_df.count()
print(f"\n[INFO] Liczba klientów: {count}")

pass

In [0]:
# Zdefiniuj schemat manualnie
# Uzupełnij typy danych dla każdego pola
customers_schema = StructType([
    StructField("customer_id", ____, False),  # IntegerType
    StructField("customer_name", ____, True),  # StringType
    StructField("email", StringType(), True),
    StructField("country", StringType(), True),
    StructField("registration_date", ____, True)  # TimestampType
])

# Wczytaj ponownie z ręcznym schematem
customers_df_manual = (
    spark.read
    .format("____")  # csv
    .option("header", "____")  # true
    .schema(____)  # customers_schema
    .load(CUSTOMERS_CSV)
)

# Wyświetl schemat
print("[INFO] Schemat z ręczną definicją:")
customers_df_manual.printSchema()

# Wyświetl przykładowe dane
display(customers_df_manual.limit(5))

---

## Zadanie 2: Import danych JSON (15 min)

### Cel:
Wczytaj dane zamówień z pliku JSON i zdefiniuj schemat manualnie.

### Instrukcje:
1. Wczytaj `orders_batch.json` z `inferSchema=True`
2. Zbadaj strukturę danych (schemat, typy)
3. Zdefiniuj schemat manualnie
4. Wczytaj ponownie z ręcznym schematem

### Oczekiwany rezultat:
- DataFrame z zamówieniami
- Schemat: order_id (int), customer_id (int), order_date (timestamp), total_amount (double), status (string)

### Wskazówki:
- JSON nie wymaga opcji `header`
- Użyj `DoubleType` dla kwot pieniężnych

In [0]:
# Przykład: Wczytanie pliku JSON z zamówieniami
# Ta komórka jest już gotowa - przeanalizuj kod i uruchom ją

orders_df = (
    spark.read
    .format("json")
    .option("multiLine", "true")
    .load(ORDERS_JSON)
)

print("=== Schema ===")
orders_df.printSchema()

print("\n=== Przykładowe dane (5 wierszy) ===")
display(orders_df.limit(5))

In [0]:
# Zdefiniuj schemat dla orders
orders_schema = StructType([
    StructField("order_id", ____, False),  # IntegerType
    StructField("customer_id", IntegerType(), True),
    StructField("order_date", ____, True),  # TimestampType
    StructField("total_amount", ____, True),  # DoubleType
    StructField("status", ____, True)  # StringType
])

# Wczytaj z ręcznym schematem
orders_df_manual = (
    spark.read
    .format("____")  # json
    .option("multiLine", "true")
    .schema(____)  # orders_schema
    .load(ORDERS_JSON)
)

# Wyświetl schemat
print("[INFO] Schemat orders z ręczną definicją:")
orders_df_manual.printSchema()

# Wyświetl przykładowe dane
display(orders_df_manual.limit(5))

---

## Zadanie 3: Import danych Parquet (10 min)

### Cel:
Wczytaj dane produktów z pliku Parquet (schemat jest wbudowany).

### Instrukcje:
1. Wczytaj `products.parquet`
2. Sprawdź schemat (Parquet zawiera schemat wbudowany)
3. Wyświetl dane
4. Policz rekordy

### Oczekiwany rezultat:
- DataFrame z produktami
- Schemat automatycznie załadowany z pliku Parquet

### Wskazówki:
- Parquet nie wymaga `inferSchema` ani ręcznego schematu

In [0]:
# Wczytaj products.parquet
products_df = (
    spark.read
    .format("____")  # parquet
    .load(PRODUCTS_PARQUET)
)

# Wyświetl schemat (Parquet zawiera wbudowany schemat)
print("[INFO] Schemat products (wbudowany w Parquet):")
products_df.printSchema()

# Wyświetl dane
print("\n[INFO] Przykładowe dane produktów:")
display(products_df.limit(5))

# Policz rekordy
product_count = products_df.count()
print(f"\n[INFO] Liczba produktów: {product_count}")

---

## Zapis danych do Delta Lake

### Cel:
Zapisz wczytane DataFrames do tabel Delta Lake dla dalszego użycia.

### Instrukcje:
1. Zapisz customers do tabeli `bronze.customers_workshop`
2. Zapisz orders do tabeli `bronze.orders_workshop`
3. Zapisz products do tabeli `bronze.products_workshop`

In [0]:
# Przykład: Zapis customers do Delta Lake
# Ta komórka jest już gotowa - przeanalizuj kod i uruchom ją

customers_table = f"{BRONZE_SCHEMA}.customers_workshop"

(
    customers_df_manual
    .write
    .format("delta")
    .mode("overwrite")
    .option("overwriteSchema", "true")
    .saveAsTable(customers_table)
)

print(f"✓ Zapisano tabelę: {customers_table}")
spark.sql(f"DESCRIBE TABLE {customers_table}").show(truncate=False)

In [0]:
# Zapisz orders do Delta Lake
orders_table = f"{BRONZE_SCHEMA}.orders_workshop"

(
    orders_df_manual
    .write
    .format("____")  # delta
    .mode("____")  # overwrite
    .option("overwriteSchema", "true")
    .saveAsTable(____)  # orders_table
)

print(f"✓ Zapisano tabelę: {orders_table}")

In [0]:
# Zapisz products do Delta Lake
products_table = f"{BRONZE_SCHEMA}.products_workshop"

(
    ____  # products_df
    .write
    .format("delta")
    .mode("overwrite")
    .option("overwriteSchema", "true")
    .saveAsTable(products_table)
)

print(f"✓ Zapisano tabelę: {products_table}")

---

## Zadanie 4: Eksploracja danych - Customers (15 min)

### Cel:
Przeprowadź szczegółową eksplorację danych klientów.

### Instrukcje:
1. Wyświetl listę kolumn i ich typy
2. Policz liczbę unikalnych klientów
3. Znajdź liczbę klientów według krajów
4. Sprawdź, czy są wartości NULL w kolumnach
5. Wygeneruj statystyki opisowe (`describe()`)

### Oczekiwany rezultat:
- Kompletna analiza eksploracyjna
- Zidentyfikowane braki danych
- Rozkład geograficzny klientów

### Wskazówki:
- Użyj: `columns`, `dtypes`, `count()`, `distinct()`, `groupBy()`, `describe()`
- Do sprawdzenia NULL: `.filter(col("column_name").isNull())`

In [0]:
# Wyświetl kolumny i typy
print("=== Kolumny i typy danych ===")
print("Kolumny:", customers_df_manual.columns)
print("\nTypy danych:", customers_df_manual.dtypes)

# TODO: Policz unikalne wartości customer_id
unique_customers = customers_df_manual.select("____").distinct().count()  # customer_id
total_customers = customers_df_manual.count()

print(f"\n=== Statystyki podstawowe ===")
print(f"Liczba wszystkich rekordów: {____}")  # total_customers
print(f"Liczba unikalnych klientów: {unique_customers}")
print(f"Duplikaty: {total_customers - unique_customers}")

In [0]:
# Policz klientów według krajów
customers_by_country = (
    customers_df_manual
    .groupBy("____")  # country
    .count()
    .orderBy("count", ascending=____)  # False
)

print("=== Rozkład klientów według krajów ===")
display(customers_by_country)

In [0]:
# Sprawdź wartości NULL w każdej kolumnie
from pyspark.sql.functions import col, sum as spark_sum

null_counts = customers_df_manual.select([
    spark_sum(col(c).____().____("int")).alias(c)  # isNull, cast
    for c in customers_df_manual.columns
])

print("=== Liczba wartości NULL w każdej kolumnie ===")
display(null_counts)

# Oblicz procent brakujących danych
total_rows = customers_df_manual.count()
print(f"\nCałkowita liczba rekordów: {total_rows}")

In [0]:
# Wygeneruj statystyki opisowe
print("=== Statystyki opisowe ===")
display(customers_df_manual.____())  # describe

---

## Zadanie 5: Eksploracja danych - Orders (15 min)

### Cel:
Przeprowadź analizę zamówień.

### Instrukcje:
1. Policz liczbę zamówień według statusu
2. Oblicz całkowitą wartość zamówień
3. Znajdź średnią, min, max wartość zamówienia
4. Sprawdź braki danych
5. Znajdź 10 najdroższych zamówień

### Oczekiwany rezultat:
- Statystyki biznesowe zamówień
- Identyfikacja problemów z danymi
- Top 10 zamówień

### Wskazówki:
- Użyj `.agg()` z funkcjami: `sum`, `avg`, `min`, `max`
- Do sortowania: `.orderBy(col("column").desc())`

In [0]:
# Policz zamówienia według statusu
orders_by_status = (
    orders_df_manual
    .groupBy("____")  # status
    .____()  # count
    .orderBy("count", ascending=False)
)

print("=== Zamówienia według statusu ===")
display(orders_by_status)

In [0]:
# Oblicz statystyki dla total_amount
from pyspark.sql.functions import sum, avg, min, max, count, round as spark_round

orders_stats = orders_df_manual.select(
    count("*").alias("total_orders"),
    spark_round(____("total_amount"), 2).alias("total_revenue"),  # sum
    spark_round(____("total_amount"), 2).alias("avg_order_value"),  # avg
    ____("total_amount").alias("min_order"),  # min
    ____("total_amount").alias("max_order")  # max
)

print("=== Statystyki zamówień ===")
display(orders_stats)

In [0]:
# Znajdź 10 najdroższych zamówień
top_orders = (
    orders_df_manual
    .orderBy(col("total_amount").____())  # desc
    .limit(____)  # 10
)

print("=== Top 10 najdroższych zamówień ===")
display(top_orders)

In [0]:
# Sprawdź braki danych w orders
null_counts_orders = orders_df_manual.select([
    spark_sum(col(c).isNull().cast("int")).alias(c) 
    for c in orders_df_manual.columns
])

print("=== Wartości NULL w zamówieniach ===")
display(null_counts_orders)

---

## Zadanie 6: Eksploracja danych - Products (10 min)

### Cel:
Przeprowadź analizę produktów.

### Instrukcje:
1. Sprawdź schemat i kolumny
2. Policz produkty według kategorii (jeśli kolumna istnieje)
3. Znajdź statystyki cenowe (jeśli kolumna price istnieje)
4. Wyświetl 10 najdroższych produktów

### Oczekiwany rezultat:
- Kompletna analiza produktów
- Rozkład kategorii
- Statystyki cenowe

### Wskazówki:
- Sprawdź dostępne kolumny przed analizą: `products_df.columns`

In [0]:
# Sprawdź schemat i kolumny
print("=== Kolumny produktów ===")
print("Kolumny:", products_df.columns)

print("\n=== Schemat produktów ===")
products_df.printSchema()

# Wyświetl przykładowe dane
print("\n=== Przykładowe dane ===")
display(products_df.limit(5))

In [0]:
# Jeśli kolumna 'category' istnieje, policz produkty według kategorii
if "category" in products_df.columns:
    products_by_category = (
        products_df
        .groupBy("____")  # category
        .count()
        .orderBy("____", ascending=False)  # count
    )
    
    print("=== Produkty według kategorii ===")
    display(products_by_category)
else:
    print("[INFO] Kolumna 'category' nie istnieje w danych")

In [0]:
# Jeśli kolumna 'price' istnieje, oblicz statystyki
if "price" in products_df.columns:
    products_stats = products_df.select(
        ____("*").alias("total_products"),  # count
        spark_round(avg("____"), 2).alias("avg_price"),  # price
        min("price").alias("min_price"),
        ____("price").alias("max_price")  # max
    )
    
    print("=== Statystyki cenowe produktów ===")
    display(products_stats)
else:
    print("[INFO] Kolumna 'price' nie istnieje w danych")

---

## Zadanie 7: Analiza jakości danych (15 min)

### Cel:
Stwórz raport jakości danych dla wszystkich trzech datasetów.

### Instrukcje:
1. Dla każdego datasetu policz:
   - Liczbę rekordów
   - Liczbę kolumn
   - Liczbę wartości NULL w każdej kolumnie
   - Procent kompletności danych
2. Zidentyfikuj duplikaty w każdym datasecie
3. Znajdź potencjalne problemy (np. negatywne wartości, nietypowe daty)

### Oczekiwany rezultat:
- Kompletny raport jakości danych
- Lista zidentyfikowanych problemów
- Rekomendacje działań naprawczych

### Wskazówki:
- Użyj `.count()`, `.distinct().count()` do wykrywania duplikatów
- Użyj filtrów do znajdowania nietypowych wartości

In [0]:
# Funkcja do analizy jakości danych
def data_quality_report(df, dataset_name):
    print(f"\n{'='*60}")
    print(f"=== Raport jakości danych: {dataset_name} ===")
    print(f"{'='*60}")
    
    # Podstawowe statystyki
    total_rows = df.count()
    total_cols = len(df.columns)
    print(f"\n📊 Podstawowe statystyki:")
    print(f"  Liczba rekordów: {total_rows}")
    print(f"  Liczba kolumn: {total_cols}")
    
    # Sprawdź duplikaty
    distinct_rows = df.____().count()  # distinct
    duplicates = total_rows - distinct_rows
    dup_pct = (duplicates / total_rows * 100) if total_rows > 0 else 0
    print(f"\n🔍 Duplikaty:")
    print(f"  Duplikaty: {____} ({dup_pct:.2f}%)")  # duplicates
    
    # Sprawdź wartości NULL
    print(f"\n❌ Wartości NULL w kolumnach:")
    null_counts = df.select([
        spark_sum(col(c).isNull().cast("int")).alias(c) 
        for c in df.columns
    ]).collect()[0].asDict()
    
    for col_name, null_count in null_counts.items():
        null_pct = (null_count / total_rows * 100) if total_rows > 0 else 0
        status = "✓" if null_count == 0 else "⚠"
        print(f"  {status} {col_name}: {null_count} ({null_pct:.2f}%)")
    
    print(f"\n{'='*60}\n")

# Uruchom raport dla każdego datasetu
data_quality_report(____, "Customers")  # customers_df_manual
data_quality_report(orders_df_manual, "Orders")
data_quality_report(____, "Products")  # products_df

In [0]:
# Sprawdź negatywne wartości w orders (total_amount < 0)
negative_orders = orders_df_manual.filter(col("____") < 0).count()  # total_amount
zero_orders = orders_df_manual.filter(col("total_amount") == ____).count()  # 0

print("=== Problemy z wartościami zamówień ===")
print(f"⚠ Zamówienia z negatywną wartością: {negative_orders}")
print(f"⚠ Zamówienia z wartością zero: {zero_orders}")

# Sprawdź nietypowe daty
from pyspark.sql.functions import year, current_date, datediff

future_orders = orders_df_manual.filter(col("order_date") > ____()).count()  # current_date
old_orders = orders_df_manual.filter(year(col("____")) < 2020).count()  # order_date

print(f"\n=== Problemy z datami ===")
print(f"⚠ Zamówienia z przyszłą datą: {future_orders}")
print(f"⚠ Zamówienia starsze niż 2020: {old_orders}")

---

## Walidacja i weryfikacja

### Checklist - Co powinieneś uzyskać:
- [ ] Wszystkie 3 datasety wczytane (customers, orders, products)
- [ ] Schematy zdefiniowane manualnie dla CSV i JSON
- [ ] Kompletna eksploracja każdego datasetu
- [ ] Raport jakości danych wygenerowany
- [ ] Zidentyfikowane problemy z danymi

### Komendy weryfikacyjne:

In [0]:
print("="*60)
print("=== WERYFIKACJA WYNIKÓW WARSZTATU ===")
print("="*60)

# Sprawdź, czy DataFrames są zdefiniowane
try:
    assert 'customers_df_manual' in locals(), "customers_df_manual nie został utworzony"
    assert 'orders_df_manual' in locals(), "orders_df_manual nie został utworzony"
    assert 'products_df' in locals(), "products_df nie został utworzony"
    
    print("\n✓ Wszystkie DataFrames utworzone")
    
    # Sprawdź liczby rekordów
    print(f"\n📊 Liczba rekordów:")
    print(f"  ✓ Customers: {customers_df_manual.count()}")
    print(f"  ✓ Orders: {orders_df_manual.count()}")
    print(f"  ✓ Products: {products_df.count()}")
    
    # Sprawdź tabele Delta Lake
    print(f"\n💾 Zapisane tabele Delta Lake:")
    tables_to_check = [
        f"{BRONZE_SCHEMA}.customers_workshop",
        f"{BRONZE_SCHEMA}.orders_workshop",
        f"{BRONZE_SCHEMA}.products_workshop"
    ]
    
    for table in tables_to_check:
        try:
            spark.table(table)
            print(f"  ✓ {table}")
        except:
            print(f"  ✗ {table} - nie istnieje")
    
    print("\n" + "="*60)
    print("✓✓✓ WARSZTAT ZAKOŃCZONY POMYŚLNIE! ✓✓✓")
    print("="*60)
    
except AssertionError as e:
    print(f"\n✗ Błąd: {e}")
except NameError as e:
    print(f"\n✗ Zmienna nie została zdefiniowana: {e}")

---

## Troubleshooting

### Problem 1: Błąd "Path does not exist"
**Objawy:**
- Błąd podczas wczytywania plików CSV/JSON/Parquet
- Komunikat: `Path does not exist: /Volumes/...`

**Rozwiązanie:**
```python
# Sprawdź, czy ścieżka istnieje
dbutils.fs.ls(DATASET_BASE_PATH)

# Upewnij się, że zmienne są poprawnie zdefiniowane
print(f"DATASET_BASE_PATH: {DATASET_BASE_PATH}")
print(f"CUSTOMERS_CSV: {CUSTOMERS_CSV}")
```

### Problem 2: Błąd typu danych podczas wczytywania
**Objawy:**
- Wartości NULL w miejscach, gdzie powinny być dane
- Niepoprawne typy kolumn (wszystko jako string)

**Rozwiązanie:**
- Zawsze definiuj schemat manualnie zamiast używać `inferSchema=True`
- Użyj odpowiednich typów: `IntegerType()`, `StringType()`, `TimestampType()`, `DoubleType()`

### Problem 3: Tabela już istnieje
**Objawy:**
- Błąd: `Table already exists`

**Rozwiązanie:**
```python
# Użyj mode="overwrite"
df.write.format("delta").mode("overwrite").saveAsTable(table_name)

# Lub usuń tabelę przed zapisem
spark.sql(f"DROP TABLE IF EXISTS {table_name}")
```

### Debugging tips:
- Użyj `.printSchema()` aby sprawdzić strukturę DataFrame
- Użyj `.explain()` aby zobaczyć plan wykonania
- Sprawdź liczbę rekordów: `df.count()`
- Wyświetl przykładowe dane: `display(df.limit(10))`

## Best Practices

### Wydajność:
- **Zawsze definiuj schemat manualnie** - `inferSchema=True` jest wolne i może dawać błędne typy
- **Używaj Parquet dla dużych datasetów** - najlepsza kompresja i wydajność
- **Partycjonuj duże tabele** według często używanych kolumn filtrujących
- **Używaj `.cache()` tylko gdy DataFrame jest wielokrotnie użyty**

### Jakość kodu:
- **Nazywaj DataFrames opisowo**: `customers_df`, `orders_clean_df`, `products_enriched_df`
- **Dodawaj komentarze** wyjaśniające logikę biznesową
- **Sprawdzaj dane po każdym kroku** transformacji
- **Używaj zmiennych dla nazw tabel i schematów** (nie hardcoduj)

### Data Quality:
- **Zawsze sprawdzaj wartości NULL** przed transformacjami
- **Wykrywaj duplikaty**: `df.distinct().count()` vs `df.count()`
- **Waliduj zakresy wartości**: negatywne ceny, przyszłe daty
- **Dokumentuj znalezione problemy** w komentarzach lub osobnym notebooku

### Governance:
- **Używaj Unity Catalog** dla zarządzania tabelami i uprawnieniami
- **Stosuj konwencję nazewnictwa**: `bronze.customers`, `silver.customers_clean`, `gold.customers_agg`
- **Separuj środowiska**: dev/staging/prod używając różnych katalogów
- **Loguj operacje**: zapisuj metadane o przetwarzanych danych

---

## Podsumowanie

### Co zostało osiągnięte:
- ✅ Skonfigurowałeś środowisko Databricks i zmienne per-user
- ✅ Wczytałeś dane z trzech różnych formatów (CSV, JSON, Parquet)
- ✅ Zdefiniowałeś schematy manualnie (best practice)
- ✅ Przeprowadziłeś szczegółową eksplorację wszystkich datasetów
- ✅ Zidentyfikowałeś problemy jakości danych
- ✅ Wygenerowałeś kompletny raport jakości danych
- ✅ Zapisałeś dane do tabel Delta Lake

### Kluczowe wnioski:
1. **Ręczne schematy > inferSchema**: Szybsze, bezpieczniejsze i przewidywalne typy danych
2. **Eksploracja przed transformacją**: Zawsze analizuj dane przed rozpoczęciem przetwarzania
3. **Jakość wymaga monitorowania**: NULL, duplikaty, outliers muszą być wykrywane systematycznie
4. **Parquet to standard lakehouse**: Wbudowany schemat, najlepsza kompresja i wydajność

### Quick Reference - Najważniejsze komendy:

| Operacja | PySpark | Notatki |
|----------|---------|---------|
| Wczytaj CSV | `spark.read.format("csv").option("header","true").schema(schema).load(path)` | Zawsze używaj ręcznego schematu |
| Wczytaj JSON | `spark.read.format("json").schema(schema).load(path)` | Opcja `multiLine=true` dla JSON arrays |
| Wczytaj Parquet | `spark.read.format("parquet").load(path)` | Schemat wbudowany |
| Zapisz do Delta | `df.write.format("delta").mode("overwrite").saveAsTable(table)` | mode: overwrite/append |
| Sprawdź NULL | `df.select([sum(col(c).isNull()).alias(c) for c in df.columns])` | Dla każdej kolumny |
| Znajdź duplikaty | `df.count() - df.distinct().count()` | Różnica = liczba duplikatów |
| Statystyki | `df.describe()` | Dla kolumn numerycznych |
| Grupowanie | `df.groupBy("col").count()` | Agregacje |

### Następne kroki:
- **Kolejny warsztat**: `02_transformations_cleaning_workshop.ipynb` - Transformacje i czyszczenie danych
- **Praktyka**: Powtórz eksplorację na własnych datasetach
- **Dokumentacja**: [Databricks Data Import Best Practices](https://docs.databricks.com/ingestion/index.html)
- **Zadanie domowe**: Wykonaj analizę eksploracyjną dodatkowego datasetu (np. transactions, events)

## Czyszczenie zasobów

Opcjonalne - usuń tabele utworzone podczas warsztatu:

In [0]:
# UWAGA: Uruchom tylko jeśli chcesz usunąć wszystkie utworzone tabele
# Te tabele mogą być potrzebne w kolejnych warsztatach!

# Odkomentuj poniższe linie aby usunąć tabele:

# spark.sql(f"DROP TABLE IF EXISTS {BRONZE_SCHEMA}.customers_workshop")
# spark.sql(f"DROP TABLE IF EXISTS {BRONZE_SCHEMA}.orders_workshop")
# spark.sql(f"DROP TABLE IF EXISTS {BRONZE_SCHEMA}.products_workshop")

# # Wyczyść cache
# spark.catalog.clearCache()

# print("✓ Zasoby zostały wyczyszczone")

print("[INFO] Czyszczenie zasobów jest zakomentowane - odkomentuj kod aby usunąć tabele")